In [7]:
import json
import glob
import os
import numpy as np
from pathlib import Path
import pandas as pd

from epilepsypcm.utils.make_df import make_df, get_df_list, concat_dfs, class_balance
from epilepsypcm.utils.outcome_params import engel_score

In [8]:
#Location
base_path = '/Users/david/Desktop/PCM/PCM_Data/' #modify for your file location


#Function to get a list of all dataframes for all positive patients, in the format [patient number, df]
df_list = get_df_list(base_path, "1")

#Code to loop through this list
#for i in range(len(df_list)):
#    print("Patient ID: ", df_list[i][0])
#    print("Patient Dataframe: ", df_list[i][1].head(3))


    
#Function to get the concatenated dataframe for all positive patients
## balance parameter can be changed to "None", "upsample", or "downsample"
all_positive_patients = concat_dfs(base_path, "1", balance = "None")

#print("Full Dataframe: ", all_positive_patients.head(3))

/Users/david/Documents/GitHub/epilepsypcm/epilepsypcm/utils/make_df.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["outcome"][df["chNames"] == channel] = 1


In [9]:
all_positive_patients

,chNames,significant,n1Zscore,n2Zscore,p2Zscore,n1Latency,n2Latency,p2Latency,flipped,outcome
2,LA3_LA4,0.0,1.396644,1.062212,-5.025081,13.0,180.0,43.0,1.0,1.0
6,LA7_LA8,0.0,1.371992,1.735958,-0.305671,17.0,131.0,47.0,1.0,0.0
7,LA8_LA9,0.0,1.990935,0.596185,0.596185,19.0,101.0,101.0,1.0,0.0
8,LA9_LA10,0.0,-2.361205,-1.335067,-0.176026,11.0,101.0,43.0,0.0,1.0
9,LAH1_LAH2,0.0,0.901176,1.048693,-1.308852,11.0,321.0,98.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
706,RPFS6_RPFS7,0.0,-0.775927,-2.579947,4.290270,51.0,285.0,195.0,0.0,0.0
707,RPFS7_RPFS8,0.0,1.769931,5.228859,-3.093837,12.0,139.0,69.0,1.0,0.0
708,RPFI1_RPFI2,0.0,1.361718,2.097774,-1.999799,14.0,338.0,278.0,1.0,0.0
709,RPFI2_RPFI3,0.0,-2.863802,-2.631576,7.996893,44.0,228.0,118.0,0.0,0.0
